In [1]:
from csoundengine import *

In [2]:
e = Engine(latency=0.05)
s = e.session()




--Csound version 6.19 (double samples) Oct 30 2023
[commit: 79e5d31bea441e5289073319ab2b1e42d01814b9]
libsndfile-1.2.0
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 512 sample-frame blocks
reading 4096-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 1024 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


In [4]:
tab = s.readSoundfile('/home/em/Lib/snd/samples/test/piano-pitchtest-Cmaj-M-N.flac')
s.defInstr('test0', r'''
kfreq = p5
outch 1, oscili:a(0.1, lag(kfreq, 0.01))
''')
s.sched('test0', 0, 2, kfreq=440)
ev = s.playSample(tab)
ev

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 .playSample=502.0003 start=7.337 dur=-1.000 p4=301 isndtab=0 istart=0.05 ifadein=0.05 ifadeout=1 kchan=1 kspeed=1 kgain=0.5 kpan=-1)

In [5]:
s.getInstr('.playSample')

Instr(.playSample, isndtab:5=0, istart:6=0, ifadein:7=0, ifadeout:8=0, kchan:9=1, speed(kspeed):10=1, gain(kgain):11=1, pan(kpan):12=0.5, ixfade:13=-1)

In [5]:
s.defInstr('test1', r'''
|kfreq|
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test1, kfreq:5=0)

In [5]:
name = 'test5'
s.defInstr(name, r'''
ibus = p5
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test5, ibus:5)

In [13]:
bus = s.assignBus(value=1000)
print(bus)
synth = s.sched(name, ibus=bus.token)
bus.automate((0, 800, 3, 500), overtake=False)
synth

Bus('control', token=7)


Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 test5=503.0009 start=72.798 dur=-1.000 p4=7)

In [9]:
synth.stop()

In [18]:
buses = [e.assignBus(value=600) for _ in range(200)]
e.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus)
    e.writeBus(bus, 500, 0.001)
    

In [18]:
s.defInstr('test', r'''
  ibus = p5
  kfreq = busin:k(ibus)
  outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')

buses = [s.assignBus(value=2000) for _ in range(200)]
s.engine.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus.token)
    bus.set(500, 0)
    

In [27]:
e.writeBus(bus.token, 1500)

In [28]:
bus.set(1600)

In [22]:
e.unschedAll()

end of section 1	 sect peak amps:  0.10000  0.00000
SECTION 1:


In [19]:
s.defInstr('sender', r'''
ibus = p5
ifreqbus = p6
kfreq = busin:k(ifreqbus)
asig vco2 0.1, kfreq
busout(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus  = p5
kgain = p6
asig = busin:a(ibus)
asig *= a(kgain)
outch 1, asig
''')


Instr(receiver, ibus:5, kgain:6)

In [22]:
bus = s.assignBus()
freqbus = s.assignBus(value=600)

chain = SynthGroup([s.sched('sender', ibus=bus, ifreqbus=freqbus),
                    s.sched('receiver', ibus=bus, kgain=0.5)])
e.writeBus(freqbus.token, 400, delay=1)
chain

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,4,5:ibus
505.0004 𝍪,0.042,-1.000,1012,1013
p1,start,dur,4,5:ibus
506.0004 𝍪,0.042,-1.000,1012,0.5
